In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import ml.generators.<whatever generator> as generator
import tensorflow as tf

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
TrainGen = generator.<whatever generator>
ValGen = generator.<whatever generator>
train_gen = TrainGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

In [ ]:
from keras.models import Model
from keras.applications.<whatever model> import <model>
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

In [ ]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [ ]:
base_model = <model>(weights='imagenet', include_top=False, input_shape=(220, 220, 3))
for layer in base_model.layers:
    layer.trainable = False
finished_model = add_new_last_layer(base_model, 1)
finished_model.compile(optimizer=adam(lr=0.0001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
finished_model.summary()

In [ ]:
mc_callback = ModelCheckpoint(filepath='tmp/<model>_pretrained_weights.hdf5', verbose=1)
finished_model.fit_generator(
    generator=train_gen.generate(),
    steps_per_epoch=train_gen.get_steps_per_epoch(),
    validation_data=val_gen.generate(),
    validation_steps=val_gen.get_steps_per_epoch(),
    epochs=20,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)